# Usar GPT-3 con Python

<div class="alert alert-warning">
    
<i class="fa fa-bug"></i>
Si no queréis perder los resultados de ejemplo, **no ejecutéis** el cuaderno de jupyter sin tener una clave propia de la API de OpenAI. Estos cuadernos solo cumplen una función ilustrativa de los códigos y formas de utilizar el modelo GPT-3 con Python. En caso de disponer de una **clave**, guardadla en un archivo **.env** para mayor seguridad como **un texto entrecomillado asignado a la variable OPENAI_API_KEY**.
</div>

## Autentificación

La API de OpenAI utiliza claves de API para la autentificación. 

<div class="alert alert-danger">
    
<i class="fa fa-exclamation-circle"></i>
Recuerda que tu clave de API es un secreto. No la compartas con otros ni la expongas en ningún código del lado del cliente (navegadores, aplicaciones). Las solicitudes de producción deben dirigirse a través de su propio servidor *backend*, donde su clave de API puede cargarse de forma segura desde una variable de entorno o un servicio de gestión de claves.

</div>

Todas las solicitudes de API deben incluir su clave de API, es importante almacenar en un documento seguro la llave. Para ello, crea un archivo nuevo `.env` para almacenarla en su interior de la siguiente forma:

`OPENAI_API_KEY = "MI_API_KEY"`

Con esto, para recuperar la clave de la API, tendremos que usar el **getenv** de `os`.

In [ ]:
# Si no tienes instalado el paquete dotenv, ejecuta esta línea
#!pip3 install python-dotenv

In [1]:
import openai
import os

from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
motores = ['ada', 'babbage', 'curie', 'davinci']
def gpt3_crea(motores, entrada, max_palabras):
    return openai.Completion.create(engine=motores, prompt=entrada, max_tokens=max_palabras)

## Funcionamiento básico

Para utilizar la API, hay que proporcionarle una entrada de texto, una instrucción (*promps*) y el motor devolverá un texto completo según el patrón y contexto dado.

Hay que tener en cuenta que los datos con los que se ha entrenado GPT-3 llegan hasta octubre de 2019, por lo que no es capaz de *conocer* datos ni proporcionar respuesta sobre hechos actuales de forma genérica. Además, el funcionamiento óptimo se consigue a través de proporcionarle ejemplos de lo que quiere que haga, depende también de la complejidad de la tarea a realizar; una buena forma de plantear los problemas para el modelo GPT-3 es escribirlo tal como lo explicarías a un adolescente.

Cuando le hacemos una petición a GPT-3, el modelo ofrece la respuesta como un **json**; normalmente, nos interesará la respuesta que ha introducido en el texto (*text*); pero también nos proporciona información imprtante como la razón por la que ha finalizado el proceso, un id de creación, el modelo y el objetivo que perseguía la petición.

In [3]:
entrada = 'Pienso, luego'

salida = gpt3_crea(motor[2], entrada, 50)
print(salida)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " de hojear su Facebook y su Twitter, que volver\u00e1n a juzgarla. Llegar\u00e1 un d\u00eda cuando hagan falta reconciliaciones y tomandole el pelo.\n\nDurante"
    }
  ],
  "created": 1624788557,
  "id": "cmpl-3FM0zI4z8spyzmQ5sivxXBQgfcyS4",
  "model": "curie:2020-05-03",
  "object": "text_completion"
}


In [4]:
print(salida['choices'][0]['text'])

 de hojear su Facebook y su Twitter, que volverán a juzgarla. Llegará un día cuando hagan falta reconciliaciones y tomandole el pelo.

Durante


## Usos de GPT-3

GPT-3 nos ofrece la posibilidad de trabajar de diferentes formas; existen ejemplos de tareas de lo que es capaz de hacer, a veces mejor y a veces peor; como generar conversaciones, corregir gramaticalmente un texto (aunque en español escribe que solo lleva tilde…), hace resumenes de un texto dado, es capaz de generar tablas a partir de un texto largo especificando una estructura y proporcionando algunos ejemplos, tiene la originalidad para sugerir nombres de productos (inventados), trabajar con juegos de palabras, utilizar emoticonos, hacer clasificaciones de tuits o análisis de sentimiento, extraer información determinada, conversiones de tiempos y géneros de un texto; crear relatos o incluso escribir un poco de código y hacer algunos cálculos.

Cuando trabajamos con la API a través del Playground, estas cualidades se describen de forma difusa: el motor trata de inferir la tarea a realizar según el tipo de entrada que le damos. En el caso de trabajar a través de código, podemos elegir directamente la tarea que debe ejecutar:
- Creación de texto (*completion*)
- Búsquedas (*search*)
- Clasificación (*classification*)
- Respuestas (*answer*)

También dispone de Archivos *Files*, que permite cargar documentos para utilizarlos en las funciones ya descritas.

### Creación de textos

`Completion` es la habilidad más genérica y sencilla con la que podemos construir una petición para GPT-3. En este caso, con una entrada sencilla de texto, el modelo devolverá una o más posibilidades de texto. Como parámetros básicos para generar una respuesta necesitamos tres: 
- **engine**: el modelo que vamos a utilizar. **Este es el único que realmente es obligatorio**
- **prompt**: la entrada de texto que va a servirle para dar una respuesta.
- **max_tokens**: que es el número máximo de palabras que generará en la petición.

Para poder afinar un poco los ajustes del modelo, existen más parámetros que pueden añadirse al cuerpo de la petición:
- **temperature**: qué temperatura de muestreo utilizar. Los valores más altos significan que el modelo será más creativo, cuanto más se acerce a 0, las respuestas estarán más definidas. OpenAI, en la documentación recomienda alterar este parámetro o *top_p*, pero no ambos a la vez.
- **top_p**: puede verse como una alternativa a la temperatura del modelo, el modelo considerará los resultados de las palabras con masa de probabilidad top_p; los valores cercanos a 0,1 solo consideran las palabras que comprenden la masa de probabilidad del 10% superior.
- **n**: número de propuestas que se generará en una petición. (Este parámetro puede suponer un problema, puede consumir rápidamente la cuota de palabras.)
- **stream**: transmite el progreso parcial. Si se establece, las palabras se enviarán como eventos del servidor  a medida que los datos estén disponibles, cuando el proceso termine, aparecerá el mensaje [DONE].
- **logprobs**: incluye las probabilidades log en las palabras más probables de logprobs, así como las palabras elegidas. Por ejemplo, si logprobs es 10, la API devolverá una lista de las 10 palabras más probables. La API siempre devolverá el logprob de la palabra muestreada, por lo que puede haber hasta logprobs + 1 elementos en la respuesta.
- **echo**: devuelve el aviso además de la finalización
- **stop**: puedes definir hasta 4 secuencias con las que la API dejará de generar más palabras. El texto devuelto no contendrá la secuencia de parada.
- **presence_penalty**: Número entre 0 y 1 que penaliza las nuevas palabras en función de si aparecen en el texto hasta el momento. Aumenta la probabilidad de que el modelo hable de nuevos temas.
- **frequency_penalty**: Número entre 0 y 1 que penaliza las nuevas palabras en función de su frecuencia existente en el texto hasta el momento. Disminuye la probabilidad de que el modelo repita la misma línea textualmente.
- **best_of**: Genera las mejores terminaciones del lado del servidor y devuelve la "mejor" (la que tiene la menor probabilidad logarítmica por palabra). Los resultados no se pueden transmitir.
-  **logit_bias**: Modifica la probabilidad de que las palabras especificadasaparezcan en la finalización. Acepta un objeto json que asigna palabras (especificados por su ID de palabra en el tokenizador GPT) a un valor de sesgo asociado de -100 a 100. Puede utilizar esta herramienta tokenizadora (que funciona tanto para GPT-2 como para GPT-3) para convertir texto en IDs de palabra. Matemáticamente, el sesgo se añade a los logits generados por el modelo antes del muestreo. El efecto exacto variará según el modelo, pero los valores entre -1 y 1 deberían disminuir o aumentar la probabilidad de selección; valores como -100 o 100 deberían dar lugar a una prohibición o selección exclusiva de la palabra correspondiente. Como ejemplo, puede pasar {"50256": -100} para evitar que se genere la palabra <|endoftext|>.

In [5]:
ej_creacion = openai.Completion.create(engine="davinci", prompt="Había una vez", max_tokens=35, n=2, temperature= 0.73, frequency_penalty=0.20)
print('Creación 1:', ej_creacion['choices'][0]['text'])
print('Creación 2:', ej_creacion['choices'][1]['text'])

Creación 1:  una mujer que fue a una tienda de animales para comprar un perro. En la tienda había muchos perros, per
Creación 2:  una mujer que no podía tener hijos. Estaba muy triste y cada vez que veía a un niño en la cal


### Búsquedas
Estas permiten realizar una búsqueda semántica sobre un conjunto de documentos, para lo que puede proporcionar una consulta, como una pregunta en lenguaje natural o una declaración, y los documentos proporcionados se puntuarán y clasificarán en función de su relación semántica con la consulta introducida.

Los documentos pueden ser palabras, frases, párrafos o incluso documentos más largos. Por ejemplo, si consultamos cuáles son *ser vivo* dentro de los documentos: ["caballo", "cemento", "casa", "pez", "pescado", "gorrión"], obtendremos una puntuación de similitud diferente para cada uno. Cuanto mayor sea la puntuación, mayor será la similitud semántica del documento con la consulta.

**La búsqueda** calcula las puntuaciones de similitud entre la consulta proporcionada y los documentos. Los documentos se pueden pasar directamente a la API si no hay más de 200. La puntuación de similitud oscila entre 0 y 300 (a veces puede ser mayor), donde una puntuación superior a 200 suele significar que el documento es semánticamente similar a la consulta.

- **documents**: se permiten hasta 200 documentos para buscar, proporcionados como una lista de textos. La longitud máxima del documento (en palabras) es de 2034 menos el número de palabras de la consulta.
- **file**: el ID de un archivo cargado que contiene los documentos a buscar; solo se puede especificar uno de ellos, *documents* o *files*.
- **query**: consulta para buscar en los documentos.
- **max_rerank**: el número máximo de documentos que se reordenan y se devuelven en la búsqueda; solo tiene efecto cuando se establece *file*.
- **return_metadata**: booleano. Permite que el modelo devuelva los metadatos, solo tiene efecto cuando se establece *file*.

In [6]:
ej_consulta = openai.Engine("davinci").search(documents=["caballo", "cemento", "casa", "pez", "pescado", "gorrión"], query="ser vivo")
print(ej_consulta['data'])

[<OpenAIObject search_result at 0x7f9080a941d0> JSON: {
  "document": 0,
  "object": "search_result",
  "score": 311.932
}, <OpenAIObject search_result at 0x7f9080a94230> JSON: {
  "document": 1,
  "object": "search_result",
  "score": 245.791
}, <OpenAIObject search_result at 0x7f9080a942f0> JSON: {
  "document": 2,
  "object": "search_result",
  "score": 262.228
}, <OpenAIObject search_result at 0x7f9080a94350> JSON: {
  "document": 3,
  "object": "search_result",
  "score": 163.434
}, <OpenAIObject search_result at 0x7f9070898b90> JSON: {
  "document": 4,
  "object": "search_result",
  "score": 331.258
}, <OpenAIObject search_result at 0x7f9070898d70> JSON: {
  "document": 5,
  "object": "search_result",
  "score": 290.064
}]


### Clasificación
Las clasificaciones aprovechan un conjunto de ejemplos etiquetados sin ajuste fino y puede utilizarse para cualquier tarea de conversión de texto en etiqueta. Al evitar el ajuste fino, elimina la necesidad de ajustar los hiperparámetros. Los ejemplos etiquetados pueden proporcionarse a través de un archivo cargado con `file`, o enumerarse explícitamente en la solicitud utilizando el parámetro `examples` para pruebas rápidas y casos de uso a pequeña escala.

- **model**: motor a utilizar para la finalización. (Obligatorio)
- **query**: consulta que solicitamos para clasificar. (Obligatorio)
- **examples**: una lista de ejemplos con etiquetas (formato lista de listas, donde la primera posición es la frase a analizar y la segunda es la etiqueta). Se normalizarán todas las etiquetas para que estén en mayúsculas. (Puedes especificar ejemplos o archivos (file), pero no los dos en la misma consulta).
- **file**: el ID del archivo cargado que contiene ejemplos de entrenamiento. (Puedes especificar ejemplos (examples) o archivos, pero no los dos en la misma consulta).
- **labels**: conjunto de categorías que se clasifican. Si no se especifica, las etiquetas candidatas se recopilarán automáticamente a partir de los ejemplos que proporcione. Se normalizarán todas las etiquetas para que estén en mayúsculas.
- **search_model**: identificación del motor a utilizar para la búsqueda.
- **temperature**: qué temperatura de muestreo utilizar. Los valores más altos significan que el modelo será más creativo, cuanto más se acerque a 0, las respuestas estarán más definidas.
- **logprobs**: incluye las probabilidades log en las palabras más probables de logprobs, así como las elegidas. Por ejemplo, si logprobs es 10, la API devolverá una lista de las 10 palabras más probables. La API devolverá el logprob de la palabra muestreada, por lo que puede haber hasta logprobs+1 elementos en la respuesta. Cuando se establece logprobs, la finalización se añadirá automáticamente en *expand* para obtener los logprobs.
- **max_examples**: número máximo de ejemplos a clasificar cuando se utiliza el archivo. Si se establece un valor alto, se mejora la precisión, pero se incrementa la latencia y el coste.
- **logit_bias**: modifica la probabilidad de que las palabras especificadasaparezcan en la finalización. Acepta un objeto json que asigna palabras (especificados por su ID de palabra en el tokenizador GPT) a un valor de sesgo asociado de -100 a 100. Puede utilizar esta herramienta tokenizadora (que funciona tanto para GPT-2 como para GPT-3) para convertir texto en IDs de palabra. Matemáticamente, el sesgo se añade a los logits generados por el modelo antes del muestreo. El efecto exacto variará según el modelo, pero los valores entre -1 y 1 deberían disminuir o aumentar la probabilidad de selección; valores como -100 o 100 deberían dar lugar a una prohibición o selección exclusiva de la palabra correspondiente. Como ejemplo, puede pasar {"50256": -100} para evitar que se genere la palabra <|endoftext|>.
- **return_prompt**: en True, el json devuelto incluirá el campo *prompt* que contiene la frase final que se utilizó para solicitar la finalización. Es útil para la depuración.
- **return_metadata**: en True mostrará los metadatos, cada entrada de documento en el JSON devuelto contendrá un campo de metadatos. Solo tiene efecto cuando existe un *file*.
- **expand**: si el nombre de objeto está en la lista, proporciona la información completa del objeto; en caso contrario, solo devuelve el ID del objeto. Admite objetos de finalización y de archivo para la expansión.

In [8]:
openai.Classification.create(
  search_model="ada", 
  model="curie",
  examples=[
    ["Son las tres de la tarde", "Neutro"],
    ["Se han acabado las hamburguesas :-(", "Negativo"],
    ["¡Hoy me siento genial! Vamonos de fiesta", "Positivo"],
    ["Me gustan la sandía muy dulce", "Positivo"],
    ["No me gusta levantarme temprano", "Negativo"]
  ],
  query="No me gustan los lunes por la mañana, tengo que ir a trabajar",
  labels=["Positivo", "Negativo", "Neutro"],
)

<OpenAIObject classification at 0x7f9080a94e30> JSON: {
  "completion": "cmpl-3FMF6bS9SjOuJo2k0x2GzoKHtb4X7",
  "label": "Neutro",
  "model": "curie:2020-05-03",
  "object": "classification",
  "search_model": "ada",
  "selected_examples": [
    {
      "document": 2,
      "label": "Positivo",
      "text": "\u00a1Hoy me siento genial! Vamonos de fiesta"
    },
    {
      "document": 1,
      "label": "Negativo",
      "text": "Se han acabado las hamburguesas :-("
    },
    {
      "document": 0,
      "label": "Neutro",
      "text": "Son las tres de la tarde"
    },
    {
      "document": 3,
      "label": "Positivo",
      "text": "Me gustan la sand\u00eda muy dulce"
    },
    {
      "document": 4,
      "label": "Negativo",
      "text": "No me gusta levantarme temprano"
    }
  ]
}

### Preguntas y respuestas
Responde a la pregunta especificada utilizando los documentos y ejemplos proporcionados. Obliga al modelo a ajustarse y responder a las preguntas realizadas, es útil para las aplicaciones que requieren generar texto veraz basado en documentación empresarial o bases de información concretas. Como contexto puede proporcionarse una lista de hasta 200 documentos.
El modelo buscará en los documentos o archivos proporcionados el contexto relevante con el que, combinando con la estructura de los ejemplos proporcionados, generará una respuesta.

- **model**: motor a utilizar para la finalización. (Obligatorio)
- **question**: pregunta que debe responder. (Obligatorio)
- **examples**: lista de pares (pregunta, respuesta) que ayudarán a orientar el modelo hacia el tono y el formato de respuesta. Se recomienda añadir de 2 a 3 ejemplos. (Obligatorio)
- **examples_context**: fragmento de texto que contenga la información contextual utilizada para generar las respuestas de los ejemplos proporcionados. (Obligatorio)
- **documents**: lista de documentos de los que debe derivarse la respuesta para la pregunta de entrada. Si se trata de una lista vacía, la pregunta se responderá basándose en los ejemplos de pregunta-respuesta. (Puedes especificar documentos o archivos (file), pero no los dos en la misma consulta).
- **file**: ID de un archivo cargado que contiene documentos para buscar. (Puedes especificar documentos (documents) o archivos, pero no los dos en la misma consulta).
- **search_model**: identificación del motor a utilizar para la búsqueda.
- **max_rerank**: número máximo de documentos a clasificar por la Búsqueda cuando se utiliza el archivo. Si se ajusta a un valor alto mejora la precisión, pero se incrementa la latencia y el coste.
- **temperature**:qué temperatura de muestreo utilizar. Los valores más altos significan que el modelo será más creativo, cuanto más se acerce a 0, las respuestas estarán más definidas.
- **logprobs**: incluye las probabilidades log en las palabras más probables de logprobs, así como las elegidas. Por ejemplo, si logprobs es 10, la API devolverá una lista de las 10 palabras más probables. La API devolverá el logprob de la palabra muestreada, por lo que puede haber hasta logprobs + 1 elementos en la respuesta. Cuando se establece logprobs, la finalización se añadirá automáticamente en *expand* para obtener los logprobs.
- **max_tokens**: número máximo de palabras permitidos para la respuesta generada.
- **stop**: puedes definir hasta 4 secuencias con las que la API dejará de generar texto. El texto devuelto no contendrá la secuencia de parada.
- **n**: cuántas respuestas generar para cada pregunta.
- **logit_bias**: modifica la probabilidad de que las palabras especificadasaparezcan en la finalización. Acepta un objeto json que asigna palabras (especificados por su ID de palabra en el tokenizador GPT) a un valor de sesgo asociado de -100 a 100. Puede utilizar esta herramienta tokenizadora (que funciona tanto para GPT-2 como para GPT-3) para convertir texto en IDs de palabra. Matemáticamente, el sesgo se añade a los logits generados por el modelo antes del muestreo. El efecto exacto variará según el modelo, pero los valores entre -1 y 1 deberían disminuir o aumentar la probabilidad de selección; valores como -100 o 100 deberían dar lugar a una prohibición o selección exclusiva de la palabra correspondiente. Como ejemplo, puede pasar {"50256": -100} para evitar que se genere la palabra <|endoftext|>.
- **return_metadata**: en True mostrará los metadatos, cada entrada de documento en el json devuelto contendrá un campo de metadatos. Solo tiene efecto cuando existe un *file*.
- **return_prompt**: en True, el JSON devuelto incluirá el campo *prompt* que contiene la palabra final que se utilizó para solicitar la finalización. Es útil para la depuración.
- **expand**: si el nombre de objeto está en la lista, proporciona la información completa del objeto; en caso contrario, solo devuelve el ID del objeto. Admite objetos de finalización y de archivo para la expansión.

In [10]:
openai.Answer.create(
  search_model="babbage", 
  model="curie",
  question="¿De qué color es la manzana?", 
  documents=["La manzana es roja.", "La manzana es verde.", "La manzana es rosa."],
  examples_context="La sidra tiene propiedades antioxidantes, eleva el colesterol bueno y ayuda a la digestión.", 
  examples=[["¿Qué propiedades tiene la sidra?","Es antioxidante."], ["¿Qué propiedades tiene la sidra?","Puede ayudar a mejorar la digestión."]],
  max_tokens=10,
  stop=["\n", "<|endoftext|>"],
)

<OpenAIObject answer at 0x7f9080aaf530> JSON: {
  "answers": [
    "De rosa."
  ],
  "completion": "cmpl-3FMJVnUM26u8BwwU49omK1ua2N0Ja",
  "model": "curie:2020-05-03",
  "object": "answer",
  "search_model": "babbage",
  "selected_documents": [
    {
      "document": 0,
      "text": "La manzana es roja. "
    },
    {
      "document": 2,
      "text": "La manzana es rosa. "
    },
    {
      "document": 1,
      "text": "La manzana es verde. "
    }
  ]
}

### Archivos

Puedes conseguir información de los archivos que tiene el sistema para analizar utilizando `openai.File.list()`. Esta función es muy útil, ya que te permitirá saber qué archivos están cargados para luego poder recuperarlos o eliminarlos. Ten en cuenta que las funciones con las que estás trabajando requieren que especifiques el id de cada uno de los archivos.

In [11]:
openai.File.list()

<OpenAIObject list at 0x7f9080aaf8f0> JSON: {
  "data": [
    {
      "bytes": 77113,
      "created_at": 1624474769,
      "filename": "documentos/covid.jsonl",
      "id": "file-xiUg3HpWjA9idRbPBiZv4lpA",
      "object": "file",
      "purpose": "answers",
      "status": "processed",
      "status_details": null
    }
  ],
  "object": "list"
}

#### Cargar archivos

Puedes cargar un archivo con los documentos que utilizará GPT-3 parabuscar información y generar una respuesta. Hay que tener en cuenta que el límite de elementos cargados por una organización o usuario está en un máximo de 1 GB, por lo que es recomendable eliminar aquellos documentos que ya no sean necesarios.

Utiliza **File.create** para generar un nuevo documento. Este documento requiere de dos hiperparámetros:
- file: Nombre del archivo **JSON** que se va a cargar.
    * Si el propósito se establece como "búsqueda" o "respuesta", cada línea es un registro json con un campo *texto* y un campo opcional de *metadatos*. Solo se utilizará el campo *text* para la búsqueda. Especialmente, cuando el propósito es "respuestas", "\n" se utiliza como delimitador para trocear el contenido del campo de texto en múltiples documentos para una coincidencia más fina.
    * Si el propósito se establece como "clasificación", cada línea es un registro json con un único ejemplo de formación con los campos *text* y *label* junto con un campo opcional de *metadata*.
    
    
- purpose: El propósito de los documentos cargados.
    * Utilice "search" para Búsqueda, "answers" para Respuestas y "classifications" para Clasificaciones; permitirá validar los archivos cargados.

In [ ]:
openai.File.create(file=open("ejemplo.jsonl"), purpose='answers')

#### Recuperar archivos
Devuelve información específica de un determinado archivo, para esto, necesitamos saber la identificación que se le ha asignado al archivo a la hora de cargarlo.

Usa **File.retrieve** para devolver un json con toda la información referida a ese documento que estás consultando: **id**.

In [12]:
openai.File.retrieve("file-xiUg3HpWjA9idRbPBiZv4lpA")

<File file id=file-xiUg3HpWjA9idRbPBiZv4lpA at 0x7f9080aafcb0> JSON: {
  "bytes": 77113,
  "created_at": 1624474769,
  "filename": "documentos/covid.jsonl",
  "id": "file-xiUg3HpWjA9idRbPBiZv4lpA",
  "object": "file",
  "purpose": "answers",
  "status": "processed",
  "status_details": null
}

#### Eliminar archivos

Cuando el espacio del que disponemos es limitado, es muy importante tener claro qué es lo que ya no necesitamos para poder eliminarlo, tan importante como conocer la forma correcta de hacer desaparecer un documento que está cargado. Tendremos que utilizar **File("").delete()** para eliminar aquellos documentos que ya no va a necesitar el modelo.

In [ ]:
openai.File("file-xiUg3HpWjA9idRbPBiZv4lpA").delete()

<div class="alert alert-info">
    
<i class="fa fa-code"></i> **Este cuaderno no ha sido generado automáticamente con GPT-3.**
<hr/>
    
**Si tienes alguna duda relacionada con estos cuadernos, puedes contactar conmigo:**
Mª Reyes R.P. (Erebyel). **[Web](https://www.erebyel.es/) • [Twitter](https://twitter.com/erebyel) • [Linkedin](https://www.linkedin.com/in/erebyel/)**.
    
<hr/>
    
<i class="fa fa-plus-circle"></i> **Fuentes:**
* ***Documentación de la Beta de OpenAI***: https://beta.openai.com/docs/introduction
</div>